# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('/datasets/data.csv')
#Смотрим первые 5 строк таблицы
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
#Общая информация о таблице:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

По первым значениям строк бросается в глаза некорректность данных в столбце 'days_employed'. 
Судя по количеству значений, пропуски есть в столбцах 'days_employed' и 'total_income'.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#Изучим подробнее столбец 'days_employed'
print('Среднее арифметическое значение стажа в днях:', data['days_employed'].mean())
print('Медианное значение стажа в днях:', data['days_employed'].median())
#найдем количество строк с отрицательным значением стажа:
days_employed_negative = 0
for days in data['days_employed']:
    if days < 0:
        days_employed_negative += 1
print('количество строк с отрицательным значением стажа:', days_employed_negative)

Среднее арифметическое значение стажа в днях: 63046.49766147338
Медианное значение стажа в днях: -1203.369528770489
количество строк с отрицательным значением стажа: 15906


Большая часть значений в столбце 'days_employed' имеют отрицательное значение, при этом среднее арифметическое значение стажа составяет больше 172 лет. Так как данные в столбце некорректные, работать с ними не представляется возможным, а в задании не стоит цели проанализировать стаж, заменим пропуски на 0, а в дальнейшем переведем значения в целочисленные.

In [5]:
data['days_employed'] = data['days_employed'].fillna(0)

In [6]:
#Изучим подробнее столбец 'total_income'
print('Среднее арифметическое значение ежемесячного дохода:', data['total_income'].mean())
print('Медианное значение ежемесячного дохода:', data['total_income'].median())

Среднее арифметическое значение ежемесячного дохода: 167422.30220817294
Медианное значение ежемесячного дохода: 145017.93753253992


Разница между средним арифметическим значением дохода и его медианным значением небольшая, но для большей репрезентативности заполним пропуски медианным значением.

In [7]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

In [8]:
data.income_type.unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [9]:
#Проверим результат замены пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В исходной таблице пропуски были в столбце со стажем и столбце с ежемесячным доходом. Пропуски в стаже были заменены на 0 ввиду некорректности данных. Пропуски в ежемесячном доходе заменены на медианное значение дохода.

### Замена типа данных

In [10]:
#Заменяем длинные дроби в столбцах 'days_employed' и 'total_income' на целочисленные значения
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
#Проверим результат
display(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Значения в столбцах 'days_employed' и 'total_income' были заменены на целочисленные значения

### Обработка дубликатов

Дубликаты могут быть в столбцах 'education', 'family_status', 'income_type' и 'purpose'

Поиск и обработка дубликатов в столбце 'education'

In [11]:
#Найдем уникальные значения столбца
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [12]:
#Приведем все значения к нижнему регистру и проверим результат
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Дубликаты удалены

Поиск и обработка дубликатов в столбце 'family_status'

In [13]:
#Найдем уникальные значения столбца
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликаты отсутствуют

Поиск и обработка дубликатов в столбце 'income_type'

In [14]:
#Найдем уникальные значения столбца
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Дубликаты отсутствуют

Поиск и обработка дубликатов в столбце 'purpose'

In [15]:
#Найдем уникальные значения столбца
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце с целью получения кредита много по-разному сформулированных целей с одинаковым смыслом. Этот столбец лучше в дальнейшем категоризировать.

Теперь проверим датафрейм на наличие полных дубликатов

In [16]:
data.duplicated().sum()

71

Обнаружена 71 строка с полностью повторяющимися данными. Учитывая то, как много разнообразных данных в каждом столбце, вероятность того, что это совпадение, крайне мала. Скорее всего, в таблице вследствие какой-то ошибки продублировались некоторые строки. Разумнее всего будет удалить их.

In [17]:
data = data.drop_duplicates().reset_index(drop=True) 
#Проверим результат
data.duplicated().sum()

0

**Вывод**

Успешно удалили дубликаты в солбце с образованием и полные дубликаты в самом датафрейме. Столбец с целью кредита требует категоризации.

### Лемматизация

In [18]:
#Добавим столбец с леммами целей кредита к каждой строке
m = Mystem()
def add_lemmas(row):
    lemma = ' '.join(m.lemmatize(row['purpose']))
    return lemma
data['purpose_lemmas'] = data.apply(add_lemmas, axis=1)
#Проверка результата
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


**Вывод**

К латафрейму добавлен столбец с леммами целей получения кредита, который понадобится при категоризации данных

### Категоризация данных

In [19]:
#Выведем уникальные значения столбца с леммами целей
data['purpose_lemmas'].value_counts()

автомобиль \n                                    972
свадьба \n                                       791
на   проведение   свадьба \n                     768
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         661
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 651
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          624
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

Можно выделить следующие категории: автомобиль, свадьба, образование, недвижимость. Строки с леммой "жилье" можно включить в категорию "недвижимость".
Добавим к датафрейму столбец с категорией:

In [20]:
#Функция для одной строки
def add_category(row):
    for word in row['purpose_lemmas'].split():
        if word == 'автомобиль':
            return 'автомобиль'
        if word == 'свадьба':
            return 'свадьба'
        if word == 'образование':
            return 'образование'
        if word == 'недвижимость' or word == 'жилье':
            return 'недвижимость'
#Применим к датафрейму
data['category'] = data.apply(add_category, axis=1)    
#Проверка результата
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба


Категоризируем клиентов на тех, у кого есть дети, тех, у кого их нет

In [21]:
#Добавление столбца 'child_id', в котором значени 0 означает отсутствие детей, а 1 - их наличие
def add_child_id(row):
    if row['children'] == 0:
        return 0
    if row['children'] > 0:
        return 1
data['child_id'] = data.apply(add_child_id, axis=1)

Также проверим, не осталось ли у нас строк без категории:

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  int64  
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
 12  purpose_lemmas    21454 non-null  object 
 13  category          21454 non-null  object 
 14  child_id          21407 non-null  float64
dtypes: float64(1), int64(7), object(7)
memory usage: 2.5+ MB


Теперь обнаружилось 47 строчек, к которым не присвоились значения 'child_id'. Это значит, что в столбце 'children' стоят отрицательные значения. Выведем их на экран.

In [23]:
data.loc[data['children'] < 0]['children']

291     -1
705     -1
742     -1
800     -1
941     -1
1363    -1
1929    -1
2073    -1
3812    -1
4198    -1
4399    -1
4539    -1
5268    -1
6008    -1
6379    -1
7184    -1
7272    -1
7609    -1
7679    -1
8220    -1
8242    -1
8552    -1
9553    -1
9562    -1
10162   -1
10350   -1
10900   -1
11090   -1
11249   -1
11598   -1
12284   -1
13760   -1
14328   -1
15114   -1
15400   -1
16093   -1
16227   -1
17054   -1
17386   -1
17614   -1
18174   -1
19048   -1
19311   -1
19360   -1
20328   -1
20650   -1
21071   -1
Name: children, dtype: int64

Сложно предположить, чем вызвана такая ошибка - у этих клиентов отсутствуют дети или в следствие технической ошибки добавился знак "-". Для таких значений добавим категорию 2 в столбце 'child_id', которая будет обозначать некорректные значения, чтобы исключить их из анализа.

In [24]:
#Обновленный код для добавления столбца 'child_id' с категорией нерепрезентативных значений
def add_child_id(row):
    if row['children'] == 0:
        return 0
    if row['children'] > 0:
        return 1
    if row['children'] < 0:
        return 2
    
data['child_id'] = data.apply(add_child_id, axis=1)

#Сразу проверим результат
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
 12  purpose_lemmas    21454 non-null  object
 13  category          21454 non-null  object
 14  child_id          21454 non-null  int64 
dtypes: int64(8), object(7)
memory usage: 2.5+ MB


**Вывод**

В датафрейм добавлен столбец с категориями по целям получения кредита (автомобиль, свадьба, образование, недвижимость).

## Шаг 3. Исследование данных

### Зависимость между наличием детей и возвратом кредита в срок

In [25]:
#Сравним долю должников среди клиентов, имеющих детей и долю должников среди бездетных клиентов
#Количество должников с детьми:
debt_child = data.loc[(data['children'] != 0) & (data['debt'] == 1)]['children'].count()
#Количество клиентов с детьми:
clients_child = data.loc[data['children'] != 0]['children'].count()
#Доля должников среди клиентов, имеющих детей:
debt_child_share = debt_child / clients_child

#Количество должников без детей:
debt_childfree = data.loc[(data['children'] == 0) & (data['debt'] == 1)]['children'].count()
#Количество клиентов без детей:
clients_childfree = data.loc[data['children'] == 0]['children'].count()
#Доля должников среди клиентов, не имеющих детей:
debt_childfree_share = debt_childfree / clients_childfree

#Выведем результаты для сравнения:
print('Доля должников среди клиентов, имеющих детей, составляет {:.1%}'.format(debt_child_share))
print('Доля должников среди клиентов, не имеющих детей, составляет {:.1%}'.format(debt_childfree_share))

Доля должников среди клиентов, имеющих детей, составляет 9.2%
Доля должников среди клиентов, не имеющих детей, составляет 7.5%


**Вывод**

Доля должников среди клиентов, имеющи детей, на 23% выше, чем среди клиентов, не имеющих детей. Разница существенная, поэтому можно говорить о том, что имеется зависимость между наличием детей и возвратом кредита в срок.

In [26]:
#Создадим сводную таблицу с группировкой по наличию/отсутствию детей
data_pivot = data.pivot_table(index='child_id', columns='debt', values='children', aggfunc='count')
display(data_pivot)

debt,0,1
child_id,,
0,13028,1063
1,6639,677
2,46,1


Для большей наглядности переименуем столбцы и строки, удалим строку с некорректными данными и добавим столбец с долей должников

In [27]:
data_pivot.drop(data_pivot.index[[2]], inplace=True)
data_pivot.columns = ['Не имел задолженность', 'Имел задолженность']
data_pivot.index = ['Не имеет детей', 'Есть дети']
data_pivot['Доля должников, %'] = 100*data_pivot['Имел задолженность'] / (data_pivot['Не имел задолженность'] + data_pivot['Имел задолженность'])

data_pivot

,Не имел задолженность,Имел задолженность,"Доля должников, %"
Не имеет детей,13028,1063,7.543822
Есть дети,6639,677,9.253691


Создали сводную таблицу с группировкой по наличию/отсутствию детей и определили долю должников по каждой группе. Результат получился точно такой же, как и при анализе без группировки данных - доля должников среди клиентов, имеющих детей, на 23% выше, чем среди клиентов, не имеющих детей.

In [28]:
display(data.head())
data_pivot1 = data.pivot_table(index='child_id', values='debt', aggfunc=['count', 'mean']).style.format({('mean', 'debt'): '{:.2%}'})
data_pivot1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,category,child_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,недвижимость,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль,1
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,недвижимость,0
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,0


,count,mean
,debt,debt
child_id,,
0,14091,7.54%
1,7316,9.25%
2,47,2.13%


### Зависимость между семейным положением и возвратом кредита в срок

In [29]:
#Сравним долю должников среди клиентов разного семейного положения
#Для начала выведем словарь с соотношением семейного статуса и его id
status_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [30]:
#Количество должников в статусе женат / замужем:
debt_married = data.loc[(data['family_status_id'] == 0) & (data['debt'] == 1)]['family_status_id'].count()
#Количество клиентов в статусе женат / замужем:
clients_married = data.loc[data['family_status_id'] == 0]['family_status_id'].count()
#Доля должников среди клиентов в статусе женат / замужем:
debt_married_share = debt_married / clients_married

#Количество должников в статусе гражданский брак:
debt_married_civ = data.loc[(data['family_status_id'] == 1) & (data['debt'] == 1)]['family_status_id'].count()
#Количество клиентов в статусе гражданский брак:
clients_married_civ = data.loc[data['family_status_id'] == 1]['family_status_id'].count()
#Доля должников среди клиентов в статусе гражданский брак:
debt_married_civ_share = debt_married_civ / clients_married_civ

#Количество должников в статусе вдовец / вдова:
debt_widow = data.loc[(data['family_status_id'] == 2) & (data['debt'] == 1)]['family_status_id'].count()
#Количество клиентов в статусе вдовец / вдова:
clients_widow = data.loc[data['family_status_id'] == 2]['family_status_id'].count()
#Доля должников среди клиентов в статусе вдовец / вдова:
debt_widow_share = debt_widow / clients_widow

#Количество должников в статусе в разводе:
debt_divorced = data.loc[(data['family_status_id'] == 3) & (data['debt'] == 1)]['family_status_id'].count()
#Количество клиентов в статусе в разводе:
clients_divorced = data.loc[data['family_status_id'] == 3]['family_status_id'].count()
#Доля должников среди клиентов в статусе в разводе:
debt_divorced_share = debt_divorced / clients_divorced

#Количество должников в статусе Не женат / не замужем:
debt_single = data.loc[(data['family_status_id'] == 4) & (data['debt'] == 1)]['family_status_id'].count()
#Количество клиентов в статусе Не женат / не замужем:
clients_single = data.loc[data['family_status_id'] == 4]['family_status_id'].count()
#Доля должников среди клиентов в статусе Не женат / не замужем:
debt_single_share = debt_single / clients_single

#Выведем результаты для сравнения:
print('Доля должников среди клиентов в статусе женат / замужем: {:.1%}'.format(debt_married_share))
print('Доля должников среди клиентов в статусе гражданский брак: {:.1%}'.format(debt_married_civ_share))
print('Доля должников среди клиентов в статусе вдовец / вдова: {:.1%}'.format(debt_widow_share))
print('Доля должников среди клиентов в статусе в разводе: {:.1%}'.format(debt_divorced_share))
print('Доля должников среди клиентов в статусе Не женат / не замужем: {:.1%}'.format(debt_single_share))

Доля должников среди клиентов в статусе женат / замужем: 7.5%
Доля должников среди клиентов в статусе гражданский брак: 9.3%
Доля должников среди клиентов в статусе вдовец / вдова: 6.6%
Доля должников среди клиентов в статусе в разводе: 7.1%
Доля должников среди клиентов в статусе Не женат / не замужем: 9.8%


**Вывод**

Можно отметить, что доля должников среди тех, кто состоит или состоял в официльном браке, ниже, чем среди тех, кто никогда не вступал в брак. Возможно, вступая в официальные отношения, люди становятся ответственнее.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [31]:
#Сперва нужно категоризировать уровни дохода
#Найдем минимальный и максимальный доход, чтобы определить диапазон:
print(data['total_income'].min())
print(data['total_income'].max())

20667
2265604


Получили диапазон от 20667 до 2265604, при этом медианный доход 145018, а средний арифметический 167422. Поделим доход на следующие уровни: низкий (менее 100 тыс.), средний (100-200 тыс. включительно) и высокий (более 200 тыс). Категории уровня дохода будут 1, 2 и 3 соответственно.

In [32]:
#Добавим столбец с категорией уровня дохода
def add_income_cat(row):
    if row['total_income'] < 100000:
        return 1
    if 100000 <= row['total_income'] <= 200000:
        return 2
    if row['total_income'] > 200000:
        return 3
data['income_cat'] = data.apply(add_income_cat, axis=1)    
#Оценим репрезентативность выбранной категории:
data['income_cat'].value_counts()

2    11925
3     5066
1     4463
Name: income_cat, dtype: int64

В каждой категории сопоставимое количество клиентов, можно выполнять дальнейший анализ

In [33]:
#Найдем долю должников среди клиентов с разными уровнями дохода
#Количество должников с низким уровнем дохода:
debt_low_income = data.loc[(data['income_cat'] == 1) & (data['debt'] == 1)]['income_cat'].count()
#Количество клиентов с низким уровнем дохода:
clients_low_income = data.loc[data['income_cat'] == 1]['income_cat'].count()
#Доля должников среди клиентов с низким уровнем дохода:
debt_low_income_share = debt_low_income / clients_low_income

#Количество должников со средним уровнем дохода:
debt_mid_income = data.loc[(data['income_cat'] == 2) & (data['debt'] == 1)]['income_cat'].count()
#Количество клиентов со средним уровнем дохода:
clients_mid_income = data.loc[data['income_cat'] == 2]['income_cat'].count()
#Доля должников среди клиентов со средним уровнем дохода:
debt_mid_income_share = debt_mid_income / clients_mid_income

#Количество должников с высоким уровнем дохода:
debt_high_income = data.loc[(data['income_cat'] == 3) & (data['debt'] == 1)]['income_cat'].count()
#Количество клиентов с высоким уровнем дохода:
clients_high_income = data.loc[data['income_cat'] == 3]['income_cat'].count()
#Доля должников среди клиентов с высоким уровнем дохода:
debt_high_income_share = debt_high_income / clients_high_income

#Выведем результаты для сравнения:
print('Доля должников среди клиентов с низким уровнем дохода: {:.1%}'.format(debt_low_income_share))
print('Доля должников среди клиентов со средним уровнем дохода: {:.1%}'.format(debt_mid_income_share))
print('Доля должников среди клиентов с высоким уровнем дохода: {:.1%}'.format(debt_high_income_share))




Доля должников среди клиентов с низким уровнем дохода: 7.9%
Доля должников среди клиентов со средним уровнем дохода: 8.6%
Доля должников среди клиентов с высоким уровнем дохода: 7.1%


**Вывод**

Корреляцию уровня дохода и возврата кредита в срок моржно считать несущественной.

### Влияние цели кредита на его возврат в срок

In [34]:
#Найдем долю должников среди клиентов с разными целями кредита:
#Количество должников, взявших кредит на покупку автомобиля:
debt_car = data.loc[(data['category'] == 'автомобиль') & (data['debt'] == 1)]['category'].count()
#Количество клиентов, взявших кредит на покупку автомобиля:
clients_car = data.loc[data['category'] == 'автомобиль']['category'].count()
#Доля должников среди клиентов, взявших кредит на покупку автомобиля:
debt_car_share = debt_car / clients_car

#Количество должников, взявших кредит на свадьбу:
debt_wedding = data.loc[(data['category'] == 'свадьба') & (data['debt'] == 1)]['category'].count()
#Количество клиентов, взявших кредит на свадьбу:
clients_wedding = data.loc[data['category'] == 'свадьба']['category'].count()
#Доля должников среди клиентов, взявших кредит на свадьбу:
debt_wedding_share = debt_wedding / clients_wedding

#Количество должников, взявших кредит на образование:
debt_education = data.loc[(data['category'] == 'образование') & (data['debt'] == 1)]['category'].count()
#Количество клиентов, взявших кредит на образование:
clients_education = data.loc[data['category'] == 'образование']['category'].count()
#Доля должников среди клиентов, взявших кредит на образование:
debt_education_share = debt_education / clients_education

#Количество должников, взявших кредит на недвижимость:
debt_real_estate = data.loc[(data['category'] == 'недвижимость') & (data['debt'] == 1)]['category'].count()
#Количество клиентов, взявших кредит на недвижимость:
clients_real_estate = data.loc[data['category'] == 'недвижимость']['category'].count()
#Доля должников среди клиентов, взявших кредит на недвижимость:
debt_real_estate_share = debt_real_estate / clients_real_estate

#Выведем результаты для сравнения:
print('Доля должников среди клиентов, взявших кредит на покупку автомобиля: {:.1%}'.format(debt_car_share))
print('Доля должников среди клиентов, взявших кредит на свадьбу: {:.1%}'.format(debt_wedding_share))
print('Доля должников среди клиентов, взявших кредит на образование: {:.1%}'.format(debt_education_share))
print('Доля должников среди клиентов, взявших кредит на недвижимость: {:.1%}'.format(debt_real_estate_share))




Доля должников среди клиентов, взявших кредит на покупку автомобиля: 9.4%
Доля должников среди клиентов, взявших кредит на свадьбу: 8.0%
Доля должников среди клиентов, взявших кредит на образование: 9.2%
Доля должников среди клиентов, взявших кредит на недвижимость: 7.2%


**Вывод**

Можно отметить, что наиболеее высокая доля должников среди клиентов, взявших кредит на покупку автомобиля и на образование. Среди клиентов, взявших кредит на недвижимость, доля должников минимальная.

## Шаг 4. Общий вывод

В ходе работы был обработан датафрейм с данными о клиентах банка: удалены пропуски и дубликаты, для численных значений выбран наиболее оптимальный формат, лемматизированы и категоризированы цели получения кредита.
При анализе зависимостей даны ответы поставленные вопросы:

*Есть ли зависимость между наличием детей и возвратом кредита в срок?*

Доля должников среди клиентов, имеющи детей, на 23% выше, чем среди клиентов, не имеющих детей. Разница существенная, поэтому можно говорить о том, что имеется зависимость между наличием детей и возвратом кредита в срок.

*Есть ли зависимость между семейным положением и возвратом кредита в срок?*

Можно отметить, что доля должников среди тех, кто состоит или состоял в официльном браке, ниже, чем среди тех, кто никогда не вступал в брак. Возможно, вступая в официальные отношения, люди становятся ответственнее.

*Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*

Корреляцию уровня дохода и возврата кредита в срок моржно считать несущественной.

*Как разные цели кредита влияют на его возврат в срок?*

Можно отметить, что наиболеее высокая доля должников среди клиентов, взявших кредит на покупку автомобиля и на образование. Среди клиентов, взявших кредит на недвижимость, доля должников минимальная.